In [1]:
import os
import librosa
import numpy as np
import pandas as pd
import librosa
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from moviepy.editor import VideoFileClip
from feature_extractor import compute_audio

2025-03-04 07:38:04.157058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-04 07:38:04.305893: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-04 07:38:04.305926: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-03-04 07:38:05.217185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

In [2]:
from moviepy.editor import AudioFileClip

In [7]:
import shutil

In [30]:
class AudioProcessor:
    def __init__(self, file_path, compute_audio):
        self.file_path = file_path
        self.audio = None
        self.sr = None
        self.segments = []
        self.features = []
        self.predictions = []
        self.summary = None
        self.compute_audio = compute_audio
        self.output_folder = 'temporary'

        
    def check_format(self):
        # Define acceptable video and audio formats
        video_formats = ('.mp4', '.avi', '.mkv', '.mov', '.flv', '.wmv')
        audio_formats = ('.mp3', '.wav', '.aac', '.flac', '.ogg', '.m4a', '.wma')

        # Check if the file is a valid video or audio format
        if self.file_path.endswith(audio_formats) or self.file_path.endswith(video_formats):
            pass
        else:
            raise ValueError('File not in a support data type. Please ensure you upload an audio',
                             'or video, with mp3, wav, acc, flac, ogg, mp4, avi, or mkv formats.') 
        

    def convert_video_to_audio(self):
        # If the file is a video, convert to audio
        if self.file_path.endswith(('.mp4', '.avi', '.mkv')):
            audio_path = self.file_path.replace('.mp4', '.wav').replace('.mkv', '.wav').replace('.avi', '.wav')
            clip = VideoFileClip(self.file_path)
            clip.audio.write_audiofile(audio_path)
            self.file_path = audio_path

    def segment_audio(self):
        # Load audio and split into 30-second segments
        #self.segments = [self.file_path]
        # Load audio file
        self.audio, self.sr = librosa.load(self.file_path, sr=None)
        
        # Duration of a 30-second segment in samples
        segment_length = 30 * self.sr
        
        # Split audio into segments
        self.segments = [
            self.audio[i:i + segment_length]
            for i in range(0, len(self.audio), segment_length)
        ]
        if len(self.segments) > 1 and len(self.segments[-2]) != len(self.segments[-1]): 
            self.segments[-2] = np.concatenate((self.segments[-2], self.segments[-1]))
            self.segments.pop()  # Remove the now redundant last segment
    
    def segment_audio_using_other_method(self):
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)
        # Load the audio file using moviepy
        audio_clip = AudioFileClip(self.file_path)
        duration = audio_clip.duration  # Total duration in seconds

        # Duration of a 30-second segment in seconds
        segment_length = 30  
        start_time = 0
        clip_number = 1

        while start_time < duration:
            end_time = min(start_time + segment_length, duration)

            # Extract the segment
            segment = audio_clip.subclip(start_time, end_time)

            # Save the segment to a file
            new_filename = f"clip{clip_number}.mp3"
            segment.write_audiofile(os.path.join(self.output_folder, new_filename))

            # Prepare for the next segment
            start_time = end_time
            clip_number += 1
        
    def extract_features_new_method(self):
        
        file_paths = [os.path.join(self.output_folder, f) for f in os.listdir(self.output_folder) if f.endswith('.mp3')]
        
        # Compute chroma, ZCR, RMS, and spectral features
        for segment in file_paths:
            segment_loaded, self.sr = librosa.load(segment, sr=None)
            self.computed = self.compute_audio(segment_loaded, self.sr)
            print('this is self.computed:', type(self.computed.y))
            print(self.computed.y)
            chroma = self.computed.compute_chroma_mean()
            zcr = self.computed.compute_zcr_mean()
            rms = self.computed.compute_rms_energy()
            centroid = self.computed.compute_spectral_centroid_mean()
            bandwidth = self.computed.compute_spectral_bandwidth_mean()
            rolloff = self.computed.compute_spectral_rolloff_mean()
            self.features.append(np.hstack([chroma, zcr, rms, centroid, bandwidth, rolloff]))
        
    def extract_features(self):
        # Compute chroma, ZCR, RMS, and spectral features
        for segment in self.segments:
            self.computed = self.compute_audio(segment, self.sr)
            #print('ewwww', self.computed.y)
            chroma = self.computed.compute_chroma_mean()
            zcr = self.computed.compute_zcr_mean()
            rms = self.computed.compute_rms_energy()
            centroid = self.computed.compute_spectral_centroid_mean()
            bandwidth = self.computed.compute_spectral_bandwidth_mean()
            rolloff = self.computed.compute_spectral_rolloff_mean()
            self.features.append(np.hstack([chroma, zcr, rms, centroid, bandwidth, rolloff]))

    def normalize_features(self):
        # Normalize the features with MinMaxScaler
        scaler = MinMaxScaler()
        self.features = scaler.fit_transform(self.features)

    def classify_segments(self):
        # Load model and classify segments
        model = load_model('deep_ann_model_2025.h5')
        arabic_maqam_mapping = {
            0: 'بيات',
            1: 'حجاز',
            2: 'راست',
            3: 'سيكاه',
            4: 'صبا',
            5: 'عجم',
            6: 'كرد',
            7: 'نهاوند'
        }
        english_maqam_mapping = {
            0: 'Bayat',
            1: 'Hejaz',
            2: 'Rast',
            3: 'Seekah',
            4: 'Saba',
            5: 'Ajam',
            6: 'Kurd',
            7: 'Nahawand' # there was an error here
        }
        translate = {
            'بيات': 'Bayat',
            'حجاز': 'Hejaz',
            'راست': 'Rast',
            'سيكاه': 'Seekah',
            'صبا': 'Saba',
            'عجم': 'Ajam',
            'كرد': 'Kurd',
            'نهاوند': 'Nahawand'
        }
        self.predictions = [
            np.argmax(model.predict(np.expand_dims(feature, axis=0)))
            for feature in self.features
        ]
        self.predictions_mapped = [english_maqam_mapping[pred] for pred in self.predictions]


    def post_process_predictions(self):
        # Apply rules for outliers and maqam switches
        pass
    
    def voting_system(self):
        np_list_predictions = np.array(self.predictions_mapped)
        unique, counts = np.unique(np_list_predictions, return_counts=True)
        self.most_frequent = unique[np.argmax(counts)]
        return self.most_frequent

    def generate_summary(self):
        # Combine results into a structured summary
        summary = []
        start_time = 0
        for i, maqam in enumerate(self.predictions_mapped):
            if i == 0 or maqam != self.predictions_mapped[i - 1]:
                if i > 0:
                    summary[-1]['end'] = f"{start_time // 60:02}:{start_time % 60:02}"
                summary.append({
                    'maqam': maqam,
                    'start': f"{start_time // 60:02}:{start_time % 60:02}",
                    'end': None
                })
            start_time += 30
        if summary:
            summary[-1]['end'] = f"{start_time // 60:02}:{start_time % 60:02}"
        self.summary = summary

        

    def clear_temp(self):
        shutil.rmtree(self.output_folder)
        

    def process(self):
        # High-level method to run all steps in sequence
        self.check_format()
        self.convert_video_to_audio()
        self.segment_audio()
        self.extract_features()
        self.normalize_features()
        self.classify_segments()
        self.voting_system()
        return self.most_frequent

    
    def process_mew(self):
        # High-level method to run all steps in sequence
        self.check_format()
        self.convert_video_to_audio()
        self.segment_audio_using_other_method()
        self.extract_features_new_method()
        self.normalize_features()
        self.classify_segments()
        self.voting_system()
        self.clear_temp()
        return self.most_frequent

In [ ]:
if __name__ == '__main__':
    file_path = 'surah55-Copy1.mp4'
    processor = AudioProcessor(file_path, compute_audio)
    summary = processor.process()
    print(summary)

In [31]:
if __name__ == '__main__':
    file_path = 'surah55-Copy1.mp4'
    processor = AudioProcessor(file_path, compute_audio)
    summary = processor.process_mew()
    print(summary)

MoviePy - Writing audio in surah55-Copy1.wav


MoviePy - Done.
MoviePy - Writing audio in temporary/clip1.mp3


MoviePy - Done.
MoviePy - Writing audio in temporary/clip2.mp3


MoviePy - Done.
MoviePy - Writing audio in temporary/clip3.mp3


MoviePy - Done.
MoviePy - Writing audio in temporary/clip4.mp3


...

MoviePy - Done.
MoviePy - Writing audio in temporary/clip21.mp3


MoviePy - Done.
MoviePy - Writing audio in temporary/clip22.mp3


MoviePy - Done.
this is self.computed: <class 'numpy.ndarray'>
[-0.27044028 -0.32897973 -0.2781739  ...  0.23899001  0.27471447
  0.21442136]
this is self.computed: <class 'numpy.ndarray'>
[0.19285628 0.25212148 0.19382603 ... 0.17136462 0.1319382  0.07277904]
this is self.computed: <class 'numpy.ndarray'>
[ 0.02797854  0.01124231 -0.00158575 ... -0.00217554 -0.00277983
 -0.0023978 ]
this is self.computed: <class 'numpy.ndarray'>
[-0.15217018 -0.17264733 -0.09977181 ...  0.04036367  0.08610441
  0.0860716 ]
this is self.computed: <class 'numpy.ndarray'>
[-0.15555495 -0.21261916 -0.20583439 ...  0.375535    0.41924697
  0.3045613 ]
this is self.computed: <class 'numpy.ndarray'>
[ 0.06537426  0.08215514  0.07181429 ... -0.28735858 -0.35122964
 -0.27935758]
this is self.computed: <class 'numpy.ndarray'>
[0.2419265  0.24942371 0.12912089 ... 0.3590496  0.38915297 0.29118752]
this is self.computed: <class 'numpy.ndarray'>
[0.08761727 0.14697751 0.16336717 ... 0.41428155 0.431162   0.3084604

2025-03-04 08:41:09.824559: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/videos/myenv_new/lib/python3.7/site-packages/cv2/../../lib64:
2025-03-04 08:41:09.824614: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-04 08:41:09.824641: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-29-3): /proc/driver/nvidia/version does not exist
2025-03-04 08:41:09.825041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appr

1/1 [==============================] - 0s 31ms/step
Bayat


بيات


In [32]:
print('your mom')

your mom
